In [ ]:
import numpy as np
import os 
import pandas as pd
import scipy as sp
from IPython.display import display, HTML

In [ ]:
'''
Compute SPO, repeated 20 times for each collection
'''
COLLECTIONS = ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'TREC8', 'TREC2001', 'R04', 'TB06', 'TB06M', 'WEB14']

for COLLECTION in COLLECTIONS:
    print('Working on {}'.format(COLLECTION))
    
    # Read real table
    CSV_TABLE =  '../../src/Tables/'+COLLECTION+'.csv'
    real_table = pd.read_csv(CSV_TABLE, sep=',', header=0, index_col=0) 
    topics = real_table.columns.values

    NUM_REP = 20
    
    # Read dataset that contains the systems chunk and their retrieved documents 
    df_campione = pd.read_pickle('../df_camp.pickle')
    df_campione.set_index('collection', inplace=True)
    df_campione_coll = df_campione.loc[COLLECTION]
    df_campione_coll.set_index('time', inplace=True)

    for rep in range(0, NUM_REP):
        print('{}, '.format(rep), end='')
        
        # Select a set of systems (forming a chunk) from the dataframe  
        rnd_chunked_systems = df_campione_coll.loc[rep]['dfcamp']

        df_total = pd.DataFrame(columns=['chunk', 'system', 'topic', 'single', 'allfive', 'single_minus_allfive'])
        
        # For each system in the chunk, compute the percentage of documents retrieved by that system only (SINGLE) and by
        # all the five systems in the chunk (ALLFIVE), then compute SINGLE - ALLFIVE as a difference between these two
        # values
        for chunk, list_of_systems in enumerate(rnd_chunked_systems):
            for system_perno in list_of_systems:
                list_of_systems_no_perno = list(list_of_systems)
                list_of_systems_no_perno.remove(system_perno)

                for topic in topics:
                    aux_df_run = df_run.loc[topic]
                    aux_df_run = aux_df_run.loc[list_of_systems]

                    doc_perno = (aux_df_run.loc[system_perno]['rank_list'])
                    doc_perno2 = (aux_df_run.loc[system_perno]['rank_list'])
                    doc_others = (aux_df_run.loc[list_of_systems_no_perno]['rank_list'].values)
                    
                    doc_perno = set(doc_perno)
                    len_doc_perno = len(doc_perno)
                    doc_perno2 = set(doc_perno2)

                    for z,d in enumerate(doc_others):
                        try:
                            d = set(d)
                        except:
                            print('\n\n perno: {} \n others:{} \n topic:{}, '.format(system_perno, list_of_systems_no_perno ,topic)  )
                            display(aux_df_run)
                        doc_perno = doc_perno.difference(d)
                        doc_perno2 = doc_perno2.intersection(d)
            
                    doc_only_him = doc_perno
                    doc_everyone = doc_perno2

                    single = len(doc_only_him) / len_doc_perno * 100
                    allfive = len(doc_everyone) / len_doc_perno * 100
                    single_minus_allfive = single - allfive

                    assert  0 <= single <= 100, "error on perc"
                    assert  0 <= allfive <= 100, "error on perc"

                    df_total.loc[len(df_total)] = [chunk, system_perno , topic, single, allfive, single_minus_allfive]

        real_table.sort_index(inplace=True)
        real_table.reindex_axis(sorted(real_table.columns), axis=1)
        
        # Save the results, every variant (SINGLE, ALLFIVE, SINGLE - ALLFIVE) in a separated file
        df_spo = df_total.copy()
        df_spo.sort_values(by=['chunk', 'system' ], inplace=True)
        df_spo = df_spo.groupby(['system', 'topic']).agg('mean').reset_index()

        df_p_single = df_spo.pivot('system', 'topic', 'single')
        df_p_single.columns = [str(x)+'_SINGLE' for x in df_p_single.columns]
        df_p_single.sort_index(inplace=True)
        df_p_single.reindex_axis(sorted(df_p_single.columns), axis=1)

        df_p_allfive = df_spo.pivot('system', 'topic', 'allfive')
        df_p_allfive.columns = [str(x)+'_ALLFIVE' for x in df_p_allfive.columns]
        df_p_allfive.sort_index(inplace=True)
        df_p_allfive.reindex_axis(sorted(df_p_allfive.columns), axis=1)

        df_p_single_minus_allfive = df_spo.pivot('system', 'topic', 'single_minus_allfive')
        df_p_single_minus_allfive.columns = [str(x)+'_SINGLEMINUSALLFIVE' for x in df_p_single_minus_allfive.columns]
        df_p_single_minus_allfive.sort_index(inplace=True)
        df_p_single_minus_allfive.reindex_axis(sorted(df_p_single_minus_allfive.columns), axis=1)
        
        df_p_single.to_csv('../../pickles/SPO_dfcamp/Table/{}_SINGLE_{}.csv'.format(COLLECTION, rep),index=True, header=True)
        df_p_allfive.to_csv('../../pickles/SPO_dfcamp/Table/{}_ALLFIVE_{}.csv'.format(COLLECTION, rep),index=True, header=True)
        df_p_single_minus_allfive.to_csv('../../pickles/SPO_dfcamp/Table/{}_SINGLEMINUSALLFIVE_{}.csv'.format(COLLECTION, rep),index=True, header=True)

print('done.')
assert False